# 受信時刻調整用ノートブック

## ライブラリのインポート

In [51]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

from lib import Util

## 設定ファイルの読み込み

In [52]:
# 設定ファイルの読み込み
with open(f"{Util.get_root_dir()}/../config/config.json", "r") as f:
    config = json.load(f)

## 受信時刻調整アルゴリズム

In [53]:
class TimeAdjuster:
    """受信時刻補正を行うクラス"""
    def __init__(self, df_dict: dict, alpha: float):
        self.df_dict = df_dict # データフレームの辞書
        self.alpha   = alpha   # 標準偏差の閾値
        self.rm_idx  = set()   # 除去する行のインデックス

    def calc_idx_time_std(self, idx: int) -> float:
        """time_dictの任意の列"""
        idx_time_list = [float(self.df_dict[key]["Time"][idx]) for key in self.df_dict.keys()]
        return np.nanstd(idx_time_list)

    def calc_idx_time_argmax(self, idx: int) -> int:
        """最大値の列を計算する"""
        idx_time_list = [float(self.df_dict[key]["Time"][idx]) for key in self.df_dict.keys()]
        return np.nanargmax(idx_time_list)

    def shift_idx_time(self, key: str, idx: int) -> None:
        """指定した行を1つ後ろにずらす"""
        df                = self.df_dict[key]      # データフレーム
        df.loc[len(df)]   = np.nan                 # 末尾に行を追加
        df.iloc[idx+1:]   = df.iloc[idx:-1].values # idx行以降を1つシフト
        df.iloc[idx]      = np.nan                 # idx行を空にする
        self.df_dict[key] = df                     # 更新

    def adjust_time(self, start) -> None:
        """受信時刻の補正を行う"""
        for idx in range(start, len(self.df_dict[list(self.df_dict.keys())[0]]["Time"])):
            # 標準偏差が閾値を超える場合
            if self.calc_idx_time_std(idx) <= self.alpha:
                continue
            # 最大値の列を計算
            argmax = self.calc_idx_time_argmax(idx)
            key    = list(self.df_dict.keys())[argmax]
            # 指定した行を1つ後ろにずらす
            self.shift_idx_time(key, idx)
            self.rm_idx.add(idx)
            return idx
        # rm_idxに含まれる行を全てのデータフレームから削除
        for key in self.df_dict.keys():
            self.df_dict[key] = self.df_dict[key].drop(self.rm_idx).reset_index(drop=True)
        return True

    def judge_df_shape(self) -> bool:
        """データフレームの形状が一致しているか判定する"""
        shape = [self.df_dict[key].shape[0] for key in self.df_dict.keys()]
        return len(set(shape)) == 1

## 受信時刻調整

In [54]:
# 共通ファイルを取得
common_file = Util.get_common_files(path_list=[f"{Util.get_root_dir()}/../data/csv-data/{field_device}/amp/" for field_device in config["FieldDevice"]["Pcap"]])

In [55]:
# 共通ファイルをループ
for file_name in tqdm(common_file):
    # データフレームを辞書に格納
    df_dict_original = {}
    for field_device in sorted(config["FieldDevice"]["Pcap"]):
        file_path = f"{Util.get_root_dir()}/../data/csv-data/{field_device}/amp/{file_name}"
        df_dict_original[field_device] = pd.read_csv(file_path, index_col=0)

    # alpha を変化させながら時間調整を行う
    for alpha in np.arange(0.001, 0.1, 0.001):
        # df_dict_original をコピーして使用（元データを変更しない）
        df_dict = {key: df.copy() for key, df in df_dict_original.items()}

        # インスタンスを生成
        ta = TimeAdjuster(df_dict=df_dict, alpha=alpha)

        # 時間調整を行う
        start = 0
        try:
            while start != True:
                start = ta.adjust_time(start)
        except:
            continue

        # データフレームの形状が一致しているか判定
        if ta.judge_df_shape():
            print(f"時間調整が完了しました: {file_name}")
            # データを保存
            for field_device in sorted(config["FieldDevice"]["Pcap"]):
                Util.create_path(path=f"{Util.get_root_dir()}/../data/adjusted-data/{field_device}/amp")
                ta.df_dict[field_device].to_csv(f"{Util.get_root_dir()}/../data/adjusted-data/{field_device}/amp/{file_name}")
            break

 25%|██▌       | 1/4 [00:00<00:02,  1.47it/s]

時間調整が完了しました: 2025-03-12T00-45-19.csv


 75%|███████▌  | 3/4 [00:02<00:00,  1.20it/s]

時間調整が完了しました: 2025-03-12T01-15-19.csv


100%|██████████| 4/4 [00:03<00:00,  1.32it/s]

時間調整が完了しました: 2025-03-12T01-30-20.csv
